In [94]:
import pandas as pd
import numpy as np
import re

file_path = r"C:\Users\adamk\Documents\wordle_research\wordle-research\data_analysis\cleaned_wordle_data_missing answers.csv"

In [95]:
df = pd.read_csv(file_path)
answers_df = pd.read_csv(r"C:\Users\adamk\Documents\wordle_research\wordle-research\data_analysis\answers.csv")

In [96]:
new_df = pd.DataFrame()

# Copy the specified columns from the original DataFrame
new_df["wordle_guesses"] = df["wordle_guesses"] if "wordle_guesses" in df else "unknown"
new_df["num_guesses"] = df["num_guesses"] if "num_guesses" in df else "unknown"
new_df["wordle_answer"] = df["wordle_answer"] if "wordle_answer" in df else "unknown"
new_df["author"] = df["author"] if "author" in df else "unknown"

# Define regex patterns
title_pattern = r'^(Wordle)\s+(\d+)'
asterisk_pattern = lambda wordle_id: rf'{wordle_id}.{{0,15}}\*'


# Function to extract Wordle title, ID, and hard mode status
def extract_wordle_info(body_text):
    # Default values
    wordle_title = "unknown"
    wordle_id = "unknown"
    hard_mode = False
    
    # Extract title and wordle_id
    match = re.search(title_pattern, body_text, re.MULTILINE)
    if match:
        wordle_title = match.group(1)
        wordle_id = match.group(2)
    
    # Check for hard mode (asterisk pattern)
    if wordle_id != "unknown":
        if re.search(asterisk_pattern(wordle_id), body_text):
            hard_mode = True
    
    return wordle_title, wordle_id, hard_mode

def get_wordle_answer(row):
    if row["wordle_answer"] != "unknown":
        return row["wordle_answer"]
    else:
        wordle_id = row["wordle_id"]  # Assuming 'wordle_id' is already computed and available
        # Search in answers_df for the row with the same id
        match = answers_df[answers_df["id"] == wordle_id]
        if not match.empty:
            return match.iloc[0]["word"].lower()
        else:
            return "unknown"

In [98]:
new_df = pd.DataFrame()

new_df["wordle_answer"] = df.apply(get_wordle_answer, axis=1)



# Display the new DataFrame
print(new_df)